# **Análise de Performance do Vasco da Gama no Campeonato Brasileiro de 2003 até 2019 - Dashboard Bootstrap**

Um simples dashboard bootstrap contendo gráficos de informações sobre o Vasco da Gama a partir da modalidade de pontos corridos do brasileirão(2003 -2019)


O dataset utilizado pode ser encontrado para download aqui: https://github.com/thelesson/Colab-Analise-de-Performace-do-Vasco-da-Gama-no-Campeonato-Brasileiro-de-2003-ate-2019
Faça o download do arquivo csv:"campeonato-brasileiro-full.csv" e importe para a raiz do projeto no Colab, acessando o menu vertical esquerdo
*Para melhor visualização, execute o código abaixo  e clique no botão de menu no lado direito e escolha a opção: Ver saída em tela cheia

In [ ]:
from IPython.display import display, HTML, IFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as rc
from IPython.display import Image
import base64
df = pd.read_csv("campeonato-brasileiro-full.csv")
# Convertendo para o html
#pd.set_option('display.max_colwidth', 10)
htmTable = df.head().to_html(classes="table table-hover table-dark table-responsive")
df = df.replace(['INTERNACIONAL','REMO','MALUTROM','PALMEIRAS','BAHIA','CRUZEIRO','PONTE PRETA',
                 'FLUMINENSE','SPORT','VASCO','SANTOS','CORINTHIANS','JUVENTUDE','FLAMENGO','GUARANI','PAYSANDU','Botafogo-RJ'],
                ['Internacional','Remo','Malutrom','Palmeiras','Bahia','Cruzeiro','Ponte Preta',
                 'Fluminense','Sport','Vasco','Santos','Corinthians','Juventude','Flamengo','Guarani','Paysandu','Botafogo-rj'])
# Criar coluna 'Ano'
df['Data'] = pd.to_datetime(df['Data'])
df['Ano'] = df['Data'].dt.year

# Eliminar as partidas antes de 2003
df = df.drop(df[df.Ano < 2003].index)

# Eliminar colunas que não serão usadas
df = df.drop(['Horário', 'Dia','Rodada','Arena','Clube 1 Estado','Clube 2 Estado','Estado Clube Vencedor'], axis = 1)

# Renomear colunas
df = df.rename(columns={"Clube 1":"Mandante","Clube 2":"Visitante","Clube 1 Gols":"Mandante Gols",
                        "Clube 2 Gols":"Visitante Gols"})

# Trocar "-" por "Empate" na coluna 'Vencedor'
df = df.replace('-','Empate')

# Criar outro dataframe
dados = df.copy()

# Eliminar partidas que o Flamengo não jogou
df = df.drop(df[(df['Mandante'] != 'Vasco') & (df['Visitante'] != 'Vasco')].index)

# Vamos dar uma olhada no dataframe
df.to_csv('novo_dataframe.csv', sep='\t', encoding='utf-8')
htmtable2 = df.head().to_html(classes="table table-hover table-dark table-responsive")

jogos = df.groupby('Ano')['Ano'].count()
jogos = pd.DataFrame(jogos)
jogos.columns = ['Jogos']
jogos.reset_index(level=0, inplace=True)
df = jogos
df.to_csv('jogos.csv', sep='\t', encoding='utf-8')
htmtable3 = df.head().to_html(classes="table table-hover table-dark table-responsive")

## Partidas como mandante ##
# Criar um dataframe para partidas como mandante
df = dados
mandante = df.copy()
mandante = mandante.drop(mandante[mandante['Mandante'] != 'Vasco'].index)

# Criar um dataframe com nº de jogos por ano
jogos_mandante = mandante.groupby('Ano')['Ano'].count()
jogos_mandante = pd.DataFrame(jogos_mandante)
jogos_mandante.columns = ['Partidas mandante']
jogos_mandante.reset_index(level=0, inplace=True)

# Dataframe com o nº de gols por edição
gols_mandante = mandante.groupby('Ano')['Mandante Gols'].sum()
gols_mandante = pd.DataFrame(gols_mandante)
gols_mandante.columns = ['Mandante Gols']
gols_mandante.reset_index(level=0, inplace=True)

# Dataframe com partidas e gols por edição
gp_mandante = jogos_mandante.merge(gols_mandante, how='left', on='Ano')

# Criar coluna com a média de gols por partida
gp_mandante['GP Mandante'] = round(gp_mandante['Mandante Gols']/gp_mandante['Partidas mandante'],2)

## Partidas como visitante ##
# Criar um dataframe para partidas como visitante
visitante = df.copy()
visitante = visitante.drop(visitante[visitante['Visitante'] != 'Vasco'].index)

# Criar um dataframe com nº de jogos por ano
jogos_visitante = visitante.groupby('Ano')['Ano'].count()
jogos_visitante = pd.DataFrame(jogos_visitante)
jogos_visitante.columns = ['Partidas visitante']
jogos_visitante.reset_index(level=0, inplace=True)

# Dataframe com o nº de gols por edição
gols_visitante = visitante.groupby('Ano')['Visitante Gols'].sum()
gols_visitante = pd.DataFrame(gols_visitante)
gols_visitante.columns = ['Visitante Gols']
gols_visitante.reset_index(level=0, inplace=True)

# Dataframe com partidas e gols por edição
gp_visitante = jogos_visitante.merge(gols_visitante, how='left', on='Ano')

# Criar coluna com a média de gols por partida
gp_visitante['GP Visitante'] = round(gp_visitante['Visitante Gols']/gp_visitante['Partidas visitante'],2)

## Todas as Partidas ##
gp_geral = gp_mandante.merge(gp_visitante, how='left', on='Ano')
gp_geral['Partidas'] = gp_geral['Partidas mandante'] + gp_geral['Partidas visitante']
gp_geral['Gols'] = gp_geral['Mandante Gols'] + gp_geral['Visitante Gols']
gp_geral['GP Geral'] = round(gp_geral['Gols']/gp_geral['Partidas'],2)
#print(gp_geral)
df = gp_geral
df.to_csv('gols.csv', sep='\t', encoding='utf-8')
htmtable4 = df.head().to_html(classes="table table-hover table-dark table-responsive")

r =[0,1,2,3,4,5,6,7,8,9,10,11,12,13]
#print(r)
nomes = ('2003','2004','2005','2006','2007','2008','2010','2011','2012','2013','2015','2017','2018','2019')

coluna1 = gp_geral['Gols']
#print(gp_geral['Gols'])
# gráfico
barWidth = 0.9
# Criando a barra de gols marcados
plt.bar(r, coluna1, color='black', width=barWidth)

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Gols")
plt.title("Gols por Ano")

# Tamanho do gráfico
plt.rcParams["figure.figsize"] = [12,6]
 
# Visualizar o gráfico
#grafico1 = plt.show()
grafico1 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico1.png')

plt.close(grafico1)
#caminho1 = Image(filename='grafico1.png') 
#print(caminho1)
#app= display(Image('grafico1.png'))
#Encoded_Image=str(base64.b64encode(Image(filename='grafico1.png').data))


coluna1 = gp_geral['GP Geral']

# gráfico
barWidth = 0.9
# Criando a barra de gols marcados por partida
plt.bar(r, coluna1, color='black', width=barWidth)

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Gols")
plt.title("Gols por Partida")

# Tamanho do gráfico
plt.rcParams["figure.figsize"] = [12,6]
 
# Visualizar o gráfico
#plt.show()
grafico2 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico2.png')

plt.close(grafico2)

coluna1 = gp_geral['Mandante Gols']/gp_geral['Gols']
coluna2 = gp_geral['Visitante Gols']/gp_geral['Gols']

# gráfico
barWidth = 0.9
# Criando a barra de gols em casa
plt.bar(r, coluna1, color='black', edgecolor='white', width=barWidth,label='Gols em Casa')
# Criando a barra de gols fora de casa
plt.bar(r, coluna2, bottom=coluna1, color='blue', edgecolor='white', width=barWidth,label='Gols Fora')

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Proporção")
plt.title("Distribuição de gols em casa e fora")
 
plt.rcParams["figure.figsize"] = [12,6]
    
# Visualizar o gráfico
plt.legend()
#plt.show()
grafico3 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico3.png')

plt.close(grafico3)

# Gols por jogo
gols = gp_geral['Gols'].sum()
partidas =  gp_geral['Partidas'].sum()
gols_partida = gols/partidas
frase1 = "O Vasco da Gama tem uma média de "+str(round(gols_partida,1))+ " gols por partida"

# Gols por jogo em casa
gols_casa = gp_geral['Mandante Gols'].sum()
gols_partida_casa =  gp_geral['Partidas mandante'].sum()
frase2 = "Em casa, o Vasco da Gama tem uma média de "+str(round(gols_partida_casa,1))+ " gols por partida"

# Gols por jogo fora de casa
gols_fora = gp_geral['Visitante Gols'].sum()
partidas_fora =  gp_geral['Partidas visitante'].sum()
gols_partida_fora = gols_fora/partidas_fora
frase3 = "Fora de casa, o Vasco da Gama tem uma média de "+str(round(gols_partida_fora,1))+ " gols por partida"

# Contar o número de vitórias
vitorias_casa=mandante.groupby('Ano')['Vencedor'].apply(lambda x: (x=='Vasco').sum()).reset_index(name='Vitórias')

# Contar o número de empates
empates_casa=mandante.groupby('Ano')['Vencedor'].apply(lambda x: (x=='Empate').sum()).reset_index(name='Empates')

# Juntar os dataframes
resultado_casa = jogos_mandante.merge(vitorias_casa, how='left', on='Ano')
resultado_casa = resultado_casa.merge(empates_casa, how='left', on='Ano')

# Ajeitar coluna
resultado_casa = resultado_casa.rename(columns={"Partidas mandante":"Jogos"}) 

# Criar coluna de derrotas
resultado_casa['Derrotas']= resultado_casa['Jogos'] - resultado_casa['Vitórias']-resultado_casa['Empates']

coluna1 = resultado_casa['Vitórias']/resultado_casa['Jogos']
coluna2 = resultado_casa['Empates']/resultado_casa['Jogos']
coluna3 = resultado_casa['Derrotas']/resultado_casa['Jogos']

# gráfico
barWidth = 0.9
# Criando a barra de vitórias
plt.bar(r, coluna1, color='green', edgecolor='white', width=barWidth,label='Vitórias')
# Criando a barra de empates
plt.bar(r, coluna2, bottom=coluna1, color='gray', edgecolor='white', width=barWidth,label='Empates')
# Criando a barra de derrotas
plt.bar(r, coluna3, bottom=[i+j for i,j in zip(coluna1, coluna2)], color='red', edgecolor='white',
        width=barWidth,label='Derrotas')

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Proporção")
plt.title("Dist. de resultados em casa")
 
# Visualizar o gráfico
plt.legend()
#plt.show()
grafico4 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico4.png')

plt.close(grafico4)

resultado_casa['% Aprov.']= round((3*resultado_casa['Vitórias']+resultado_casa['Empates'])/(3*resultado_casa['Jogos']),2)
#print(resultado_casa)
df = resultado_casa
df.to_csv('resultado_casa.csv', sep='\t', encoding='utf-8')
htmtable5 = df.head().to_html(classes="table table-hover table-dark table-responsive")

coluna1 = resultado_casa['% Aprov.']

# gráfico
barWidth = 0.9
# Criando a barra de aproveitamento
plt.bar(r, coluna1, color='black', width=barWidth)

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Aproveitamento")
plt.title("Aproveitamento em casa")

plt.rcParams["figure.figsize"] = [12,6]
 
# Visualizar o gráfico
#plt.show()
grafico5 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico5.png')

plt.close(grafico5)

# Contar o número de vitórias
vitorias_fora=visitante.groupby('Ano')['Vencedor'].apply(lambda x: (x=='Vasco').sum()).reset_index(name='Vitórias')

# Contar o número de empates
empates_fora=visitante.groupby('Ano')['Vencedor'].apply(lambda x: (x=='Empate').sum()).reset_index(name='Empates')

# Juntar os dataframes
resultado_fora = jogos_visitante.merge(vitorias_fora, how='left', on='Ano')
resultado_fora = resultado_fora.merge(empates_fora, how='left', on='Ano') 

# Ajeitar coluna
resultado_fora = resultado_fora.rename(columns={"Partidas visitante":"Jogos"})

# Criar coluna de derrotas
resultado_fora['Derrotas']= resultado_fora['Jogos'] - resultado_fora['Vitórias']-resultado_fora['Empates']

coluna1 = resultado_fora['Vitórias']/resultado_fora['Jogos']
coluna2 = resultado_fora['Empates']/resultado_fora['Jogos']
coluna3 = resultado_fora['Derrotas']/resultado_fora['Jogos']

# gráfico
barWidth = 0.9
# Criando a barra de vitórias
plt.bar(r, coluna1, color='green', edgecolor='white', width=barWidth,label='Vitórias')
# Criando a barra de empates
plt.bar(r, coluna2, bottom=coluna1, color='gray', edgecolor='white', width=barWidth,label='Empates')
# Criando a barra de derrotas
plt.bar(r, coluna3, bottom=[i+j for i,j in zip(coluna1, coluna2)], color='red', edgecolor='white',
        width=barWidth,label='Derrotas')

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Proporção")
plt.title("Dist. de resultados fora de casa")
 
# Visualizar o gráfico
plt.legend()
#plt.show()
grafico6 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico6.png')

plt.close(grafico6)

resultado_fora['% Aprov.']= round((3*resultado_fora['Vitórias']+resultado_fora['Empates'])/(3*resultado_fora['Jogos']),2)

coluna1 = resultado_fora['% Aprov.']

# gráfico
barWidth = 0.9
# Criando a barra de aproveitamento
plt.bar(r, coluna1, color='black', width=barWidth)

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Aproveitamento")
plt.title("Aproveitamento fora de casa")

plt.rcParams["figure.figsize"] = [12,6]
 
# Visualizar o gráfico
#plt.show()
grafico7 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico7.png')

plt.close(grafico7)

# Deletar colunas de aproveitamento
resultado_casa = resultado_casa.drop(['% Aprov.'], axis = 1)
resultado_fora = resultado_fora.drop(['% Aprov.'], axis = 1)

# Deletar coluna repetida
resultado_casa = resultado_casa.drop(['Jogos'], axis = 1)
resultado_fora = resultado_fora.drop(['Jogos'], axis = 1)

# Renomear colunas
resultado_casa = resultado_casa.rename(columns={"Vitórias":"Vit casa","Empates":"Emp casa","Derrotas":"Der casa"})
resultado_fora = resultado_fora.rename(columns={"Vitórias":"Vit fora","Empates":"Emp fora","Derrotas":"Der fora"})

# Juntar dataframes
resultado = jogos.merge(resultado_casa, how='left', on='Ano')
resultado = resultado.merge(resultado_fora, how='left', on='Ano')

# Coluna de Pontos
resultado['Pts'] = 3*(resultado['Vit casa']+resultado['Vit fora'])+resultado['Emp casa']+resultado['Emp fora']

# Coluna com a proporção de pontos conquistados em casa
resultado['% Pts casa']= round((3*resultado['Vit casa']+resultado['Emp casa'])/resultado['Pts'],2)

# Coluna com a proporção de pontos conquistados fora de casa
resultado['% Pts fora']= round((3*resultado['Vit fora']+resultado['Emp fora'])/resultado['Pts'],2)

resultado['Vit'] = resultado['Vit casa'] + resultado['Vit fora']
resultado['Der'] = resultado['Der casa'] + resultado['Der fora']
resultado['Emp'] = resultado['Emp casa'] + resultado['Emp fora']

coluna1 = resultado['Vit']/resultado['Jogos']
coluna2 = resultado['Emp']/resultado['Jogos']
coluna3 = resultado['Der']/resultado['Jogos']

# gráfico
barWidth = 0.9
# Criando a barra de vitórias
plt.bar(r, coluna1, color='green', edgecolor='white', width=barWidth,label='Vitórias')
# Criando a barra de empates
plt.bar(r, coluna2, bottom=coluna1, color='gray', edgecolor='white', width=barWidth,label='Empates')
# Criando a barra de derrotas
plt.bar(r, coluna3, bottom=[i+j for i,j in zip(coluna1, coluna2)], color='red', edgecolor='white',
        width=barWidth,label='Derrotas')

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Proporção")
plt.title("Dist. de resultados")
 
# Visualizar o gráfico
plt.legend()
#plt.show()
grafico8 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico8.png')

plt.close(grafico8)

resultado['% Aprov.']= round((3*resultado['Vit']+resultado['Emp'])/(3*resultado['Jogos']),2)

coluna1 = resultado['% Aprov.']

# gráfico
barWidth = 0.9
# Criando a barra de aproveitamento
plt.bar(r, coluna1, color='black', width=barWidth)

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Aproveitamento")
plt.title("Aproveitamento de pontos")

plt.rcParams["figure.figsize"] = [12,6]
 
# Visualizar o gráfico
#plt.show()
grafico9 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico9.png')

plt.close(grafico9)

coluna1 = resultado['Pts']

# gráfico
barWidth = 0.9
# Criando a barra de pontos conquistados
plt.bar(r, coluna1, color='black', width=barWidth)

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Pontos")
plt.title("Pontos Conquistados")

plt.rcParams["figure.figsize"] = [12,6]
 
# Visualizar o gráfico
#plt.show()
grafico10 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico10.png')

plt.close(grafico10)

coluna1 = resultado['% Pts casa']
coluna2 = resultado['% Pts fora']

# gráfico
barWidth = 0.9
# Criando a barra de pontos conquistados em casa
plt.bar(r, coluna1, color='blue', edgecolor='white', width=barWidth,label='% Pts Casa')
# Criando a barra de pontos conquistados fora de casa
plt.bar(r, coluna2, bottom=coluna1, color='black', edgecolor='white', width=barWidth,label='% Pts Fora')

# Eixo X
plt.xticks(r, nomes)
plt.xlabel("Anos")
plt.ylabel("Proporção")
plt.title("Dist. de pontos conquistados em casa e fora")
 
plt.rcParams["figure.figsize"] = [12,6]
    
# Visualizar o gráfico
plt.legend()
#plt.show()
grafico11 =plt.savefig('/usr/local/share/jupyter/nbextensions/grafico11.png')
#grafico11b =plt.savefig('grafico11.png')
plt.close(grafico11)


display(HTML('<head>'+
             '<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" integrity="sha384-JcKb8q3iqJ61gNV9KGb8thSsNjpSL0n8PARn9HuZOnIxN0hoP+VmmDGMN5t9UJ0Z" crossorigin="anonymous">'+
             '<style>.conteudo{background:black;}.dataframe{background:black;border-collapse: inherit;} .table.dataframe > tbody > tr:hover{background-color:black!important} .table.dataframe tbody tr:nth-child(odd){background-color:black!important}</style>'
  '</head>'+
   '<body><div class="container"><div class="row"><div class="col-md-12">'+
  '<div class="jumbotron jumbotron-fluid">'+'<div class="container">'+
    '<h1 class="display-4">Análise de Performance do Vasco da Gama no Campeonato Brasileiro de 2003 até 2019</h1>'+
    '<p class="lead">Analisando o desempenho do Clube de Regatas Vasco da Gama no Campeonato Brasileiro, a partir da modalidade de pontos corridos(2003) até 2019. O dataset utilizado pode ser encontrado para download nesse repositório:<a href="https://github.com/thelesson/Colab-Analise-de-Performace-do-Vasco-da-Gama-no-Campeonato-Brasileiro-de-2003-ate-2019">GitHub</a>. Faça o download do arquivo csv: "campeonato-brasileiro-full.csv" e importe para a raiz do projeto no Colab, acessando o menu vertical esquerdo</p>'+
  '</div></div>'+
'</div></div><br/>'+'<div class="row">'+
'<div class="col-md-12">'+'<div class="card">'+
  '<div class="card-header">'+
    'Dados 2000 até 2019'
  +'</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title"> 5 linhas iniciais contendo os dados dos jogos do brasileirão de 2000 até 2019</h5>'
  +htmTable+
    '<a href="#" class="btn btn-primary">Veja a tabela completa aqui</a>'+
  '</div>'+
'</div>'+'</div></div><br/><div class="row">'+
'<div class="col-md-6">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Novo dataset contendo apenas os jogos do Vasco da Gama'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Listando os 5 primeiros dados da tabela</h5>'+
    htmtable2+
    '<a href="#" class="btn btn-primary">Ver tabela completa</a>'+
    '</div>'+
'</div></div>'+
'<div class="col-md-6">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Jogos do Vasco da Gama no Campeonato Brasileiro 2003 - 2019'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Listando a quantidade de jogos do Vasco em cada ano no Brasileirão</h5>'+
    htmtable3+
    '<a href="#" class="btn btn-primary">Ver tabela completa</a>'+
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gols marcados pelo Vasco da Gama no Campeonato Brasileiro 2003 - 2019'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Listando a quantidade de gols do Vasco como mandante, visitante e totais</h5>'+
    htmtable4+
    '<a href="#" class="btn btn-primary">Ver tabela completa</a>'+
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"><div class="col-md-6">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de Gols marcados pelo Vasco da Gama no Campeonato Brasileiro 2003 - 2019'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Listando a quantidade total de gols do Vasco em cada ano</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico1.png" alt="Total de Gols">'+
    '</div>'+
'</div></div>'+
'<br/>'+
'<div class="col-md-6">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico da média de  Gols marcados pelo Vasco da Gama no Campeonato Brasileiro 2003 - 2019, por partida'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Listando a média de gols do Vasco em cada ano, por partida</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico2.png" alt="Total de Gols">'+
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Distribuição dos gols em casa e fora de casa feitos pelo Vasco da Gama no Campeonato Brasileiro 2003 - 2019'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Comparando a quantidade de gols do Vasco feitos como mandante e como visitante</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico3.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'+
  '<h1>'+frase1+'</h1><br/><h1>'+frase2+'</h1><br/><h1>'+frase3+'</h1></div></div>'+
  '<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de desempenho do Vasco da Gama jogando em casa'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Distribuição do desempenho do Vasco jogando em casa de 2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico4.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row">'+
'<div class="col-md-12">'+'<div class="card">'+
  '<div class="card-header">'+
    'Aproveitamento de pontos em casa'
  +'</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Tabela contendo a relação de aproveitamento do Vasco da Gama em cada ano do Campeonato Brasileiro - 2003 -2019</h5>'
  +htmtable5+
    '<a href="#" class="btn btn-primary">Veja a tabela completa aqui</a>'+
  '</div>'+
'</div>'+'</div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de aproveitamento do Vasco em casa'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico do desempenho do Vasco jogando em casa de 2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico5.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de desempenho do Vasco jogando fora de casa'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico do desempenho do Vasco jogando fora de casa - 2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico6.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de aproveitamento dos pontos do Vasco jogando fora de casa'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico do aproveitamento do pontos do  Vasco jogando fora de casa - 2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico7.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de aproveitamento total do Vasco em cada ano'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico do aproveitamento total Vasco  2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico8.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Gráfico de aproveitamento dos pontos totais do Vasco em cada ano'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico do aproveitamento dos pontos totais Vasco  2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico9.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Grafico de exibição dos pontos conquistados pelo Vasco da Gama em cada ano'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico do exibição dos pontos conquistados pelo Vasco  2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico10.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'+
'<br/><div class="row"> <div class="col-md-12">'
+'<div class="card">'+
  '<div class="card-header">'+
    'Grafico Comparativo de pontos conquistados dentro e fora de casa pelo Vasco da Gama'+
  '</div>'+
  '<div class="card-body">'+
    '<h5 class="card-title">Gráfico de exibição comparativa dos pontos conquistados dentro e fora de casa pelo Vasco  2003 - 2019</h5>'+
    '<img style="width:100%" src="/nbextensions/grafico11.png" alt="Total de Gols">'
    +
    '</div>'+
'</div></div></div>'
'<script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous"></script>'+
'<script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js" integrity="sha384-UO2eT0CpHqdSJQ6hJty5KVphtPhzWj9WO1clHTMGa3JDZwrnQq4sF86dIHNDz0W1" crossorigin="anonymous"></script>'+
'<script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js" integrity="sha384-JjSmVgyd0p3pXB1rRibZUAYoIIy6OrQ6VrjIEaFf/nJGzIxFDsf4x0xIM+B07jRM" crossorigin="anonymous"></script>'+
'</body></html>'))
